In [190]:
import pandas as pd # importing pandas for dataframe  
import io #IO library so we can treat a string as a file
import re #Python library to work with regex
import math


# Creating table with the seperator ;
data =  'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'
#turning the string into a file format
data = io.StringIO(data)
df = pd.read_csv(data, sep=";") # reads in the csv
df # shows the data frame 

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


In [191]:
#Updating the Null values of FlightCodes 

# We could've use the .interpolate()(which finds the mean of the previous + next record of a NaN value and and fills the NaN with mean value)
# function and it would've given us the same answer however, I assumed that this table is not static and I wanted to take care of the edge cases
# where the first row might be NaN.
lis = list(df['FlightCodes']) # turning the column into a list
for x in range(len(lis)):
  if (math.isnan(lis[0]) == True): # takes care of the edge case if the first record is NaN and the second record is NOT NaN (this would make the function more complicated)
    lis[x] = lis[x+1] - 10
  elif (math.isnan(lis[x]) == True): # adds 10 
    lis[x] = lis[x-1] + 10

df['FlightCodes'] = lis # setting the list as the new column of FlightCodes
df = df.astype({"FlightCodes": int}) # changing the from floating point to int64 type 
df


,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055,london_MONTreal


In [192]:

# making the to and from column more readable 
new = df["To_From"].str.split("_", n = 1, expand = True)#splits on '_'
df["To"] = new[0] # setting the first new column as the To column 
df["From"] = new[1]# setting the second new column as the To column 
df.drop(columns =["To_From"], inplace = True) # drops the old To_From column

#Function to clean up the names of the cities
def clean_city (city):
    city = city.lower()
    city = city.capitalize()
    return city

# applies the function clean_city to every record in the To and From column
df['To']= df['To'].apply(clean_city) 
df['From']= df['From'].apply(clean_city)
df






,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,Waterloo,Newyork
1,<Air France> (12),[],20025,Montreal,Toronto
2,(Porter Airways. ),"[60, 22, 87]",20035,Calgary,Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa,Vancouver
4,""".\.Lufthansa.\.""","[12, 33]",20055,London,Montreal


In [193]:
#cleans up the Airline Code by removing punctuation, brackets and numbers 
def clean_airline (text):
  text = re.sub(r'([z0-9_])+','',text)#removes numbers
  text = re.sub(r"[^\w\s]", '', text)#remove punctuation

  return text

#applies the function to the Airline Code column
df['Airline Code']= df['Airline Code'].apply(clean_airline)
df


,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,Waterloo,Newyork
1,Air France,[],20025,Montreal,Toronto
2,Porter Airways,"[60, 22, 87]",20035,Calgary,Ottawa
3,Air France,"[78, 66]",20045,Ottawa,Vancouver
4,Lufthansa,"[12, 33]",20055,London,Montreal


In [ ]:
#mock SQL query on the above table to find all flights leaving from Waterloo

Select * from data 
where data.From = 'Waterloo'